In [ ]:
import cv2
import numpy as np

# ── Hyperparameters ──
mask_video_path = "masks.mp4"
output_video_path = "output.mp4"
save_path = "output_bbox.mp4"
box_thickness = 4  # line thickness in pixels

In [2]:
def get_bounding_box(mask_frame):
    """Return (x_min, y_min, x_max, y_max) from a binary mask frame, or None if no white pixels."""
    gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY) if mask_frame.ndim == 3 else mask_frame
    ys, xs = np.where(gray > 127)
    if len(xs) == 0:
        return None
    return int(xs.min()), int(ys.min()), int(xs.max()), int(ys.max())

In [3]:
mask_cap = cv2.VideoCapture(mask_video_path)
out_cap = cv2.VideoCapture(output_video_path)

fps = out_cap.get(cv2.CAP_PROP_FPS)
w = int(out_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(out_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total = int(out_cap.get(cv2.CAP_PROP_FRAME_COUNT))

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(save_path, fourcc, fps, (w, h))

yellow = (0, 255, 255)  # BGR
count = 0

while True:
    ret_m, mask_frame = mask_cap.read()
    ret_o, out_frame = out_cap.read()
    if not ret_m or not ret_o:
        break

    bbox = get_bounding_box(mask_frame)
    if bbox is not None:
        x1, y1, x2, y2 = bbox
        cv2.rectangle(out_frame, (x1, y1), (x2, y2), yellow, box_thickness)

    writer.write(out_frame)
    count += 1

mask_cap.release()
out_cap.release()
writer.release()
print(f"Done — {count}/{total} frames written to {save_path}")

Done — 496/496 frames written to output_bbox.mp4
